ETAP 1

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18

In [ ]:
# Ustawienia
batch_size = 64
num_classes = 50
learning_rate = 0.01
epochs = 25

In [ ]:
# Przygotowanie danych
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Wybieram pierwsze 50 klas
trainset.targets = torch.tensor(trainset.targets)
train_indices = torch.where(trainset.targets < num_classes)[0]
trainloader = torch.utils.data.DataLoader(torch.utils.data.Subset(trainset, train_indices), batch_size=batch_size, shuffle=True)

In [ ]:
# Wybieram pierwsze 50 klas
trainset.targets = torch.tensor(trainset.targets)
train_indices = torch.where(trainset.targets < num_classes)[0]
trainloader = torch.utils.data.DataLoader(torch.utils.data.Subset(trainset, train_indices), batch_size=batch_size, shuffle=True)

<ipython-input-6-67d21b9333dd>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  trainset.targets = torch.tensor(trainset.targets)


In [ ]:
testset.targets = torch.tensor(testset.targets)
test_indices = torch.where(testset.targets < num_classes)[0]
testloader = torch.utils.data.DataLoader(torch.utils.data.Subset(testset, test_indices), batch_size=batch_size, shuffle=False)

In [ ]:
# Definicja modelu
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.feature_extractor = resnet18(pretrained=False)
        self.feature_extractor.fc = nn.Linear(self.feature_extractor.fc.in_features, num_classes)

    def forward(self, x):
        return self.feature_extractor(x)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN(num_classes=num_classes).to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
# Funkcja kosztu i optymalizator
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Trenowanie modelu
def train_model():
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(trainloader)}")


In [ ]:
# Testowanie modelu
def test_model():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy: {100 * correct / total}%")

train_model()
test_model()

Epoch 1/25, Loss: 3.66040355531151
Epoch 2/25, Loss: 3.118846065857831
Epoch 3/25, Loss: 2.7513762648453186
Epoch 4/25, Loss: 2.392432375942045
Epoch 5/25, Loss: 2.0411629234738364
Epoch 6/25, Loss: 1.7323831908233331
Epoch 7/25, Loss: 1.437927409511088
Epoch 8/25, Loss: 1.154983504196567
Epoch 9/25, Loss: 0.8617766401194551
Epoch 10/25, Loss: 0.5663968526265201
Epoch 11/25, Loss: 0.3319317531555205
Epoch 12/25, Loss: 0.20695663510304887
Epoch 13/25, Loss: 0.15731506622241587
Epoch 14/25, Loss: 0.13676562901500547
Epoch 15/25, Loss: 0.13056816223561002
Epoch 16/25, Loss: 0.11831398557661973
Epoch 17/25, Loss: 0.09466334768687673
Epoch 18/25, Loss: 0.08481829334050417
Epoch 19/25, Loss: 0.07685886888676668
Epoch 20/25, Loss: 0.10185854148615123
Epoch 21/25, Loss: 0.0748552166299461
Epoch 22/25, Loss: 0.07645841454610686
Epoch 23/25, Loss: 0.08880160553881046
Epoch 24/25, Loss: 0.05287027526039468
Epoch 25/25, Loss: 0.05417009436315797
Accuracy: 52.94%


In [ ]:
torch.save(model.state_dict(), "model.pth")

In [ ]:
# Użycie wytrenowanego modelu jako ekstraktora cech
class FeatureExtractor(nn.Module):
    def __init__(self, model):
        super(FeatureExtractor, self).__init__()
        self.feature_extractor = nn.Sequential(*list(model.feature_extractor.children())[:-1])  # Bez ostatniej warstwy FC

    def forward(self, x):
        x = self.feature_extractor(x)
        return x.view(x.size(0), -1)

# Podział zbioru na A i B
half_classes = num_classes // 2
trainset_A_indices = np.where(np.array(trainset.targets) < half_classes)[0]
trainset_B_indices = np.where(np.array(trainset.targets) >= half_classes)[0]

trainloader_A = torch.utils.data.DataLoader(torch.utils.data.Subset(trainset, trainset_A_indices), batch_size=batch_size, shuffle=True)
trainloader_B = torch.utils.data.DataLoader(torch.utils.data.Subset(trainset, trainset_B_indices), batch_size=batch_size, shuffle=True)

testset_B_indices = np.where(np.array(testset.targets) >= half_classes)[0]
testloader_B = torch.utils.data.DataLoader(torch.utils.data.Subset(testset, testset_B_indices), batch_size=batch_size, shuffle=False)

# Trenujemy ekstraktor cech tylko na zbiorze A
model.eval()
feature_extractor_A = FeatureExtractor(model).to(device)

def extract_features(loader, feature_extractor):
    features, labels = [], []
    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            feat = feature_extractor(inputs).cpu().numpy()
            features.append(feat)
            labels.append(targets.cpu().numpy())
    return np.vstack(features), np.hstack(labels)

train_features_A, train_labels_A = extract_features(trainloader_A, feature_extractor_A)
test_features_A, test_labels_A = extract_features(testloader, feature_extractor_A)

# Testowanie dla różnych wielkości podzbiorów na zbiorze A
def sample_subset(features, labels, samples_per_class):
    sampled_features, sampled_labels = [], []
    unique_classes = np.unique(labels)
    for cls in unique_classes:
        indices = np.where(labels == cls)[0]
        chosen_indices = np.random.choice(indices, min(samples_per_class, len(indices)), replace=False)
        sampled_features.append(features[chosen_indices])
        sampled_labels.append(labels[chosen_indices])
    return np.vstack(sampled_features), np.hstack(sampled_labels)

def knn_classification(train_features, train_labels, test_features, test_labels, k=5):
    knn = KNeighborsClassifier(n_neighbors=k, metric='cosine')
    knn.fit(train_features, train_labels)
    predictions = knn.predict(test_features)
    accuracy = np.mean(predictions == test_labels)
    print(f'Accuracy dla k={k}: {accuracy * 100:.2f}%')

for subset_size in [1, 5, 10]:
    sampled_train_features_A, sampled_train_labels_A = sample_subset(train_features_A, train_labels_A, subset_size)
    print(f'Czesc A: Klasyfikacja dla podzbiorow o rozmiarze {subset_size}')
    knn_classification(sampled_train_features_A, sampled_train_labels_A, test_features_A, test_labels_A, k=5)


train_features_B, train_labels_B = extract_features(trainloader_B, feature_extractor_A)
test_features_B, test_labels_B = extract_features(testloader_B, feature_extractor_A)

# Klasyfikacja KNN na zbiorze B
for subset_size in [1, 5, 10]:
    sampled_train_features_B, sampled_train_labels_B = sample_subset(train_features_B, train_labels_B, subset_size)
    print(f'Czesc B: Klasyfikacja dla podzbiorow o rozmiarze {subset_size}')
    knn_classification(sampled_train_features_B, sampled_train_labels_B, test_features_B, test_labels_B, k=5)

Czesc A: Klasyfikacja dla podzbiorow o rozmiarze 1
Accuracy dla k=5: 7.60%
Czesc A: Klasyfikacja dla podzbiorow o rozmiarze 5
Accuracy dla k=5: 24.34%
Czesc A: Klasyfikacja dla podzbiorow o rozmiarze 10
Accuracy dla k=5: 27.14%
Czesc B: Klasyfikacja dla podzbiorow o rozmiarze 1
Accuracy dla k=5: 9.84%
Czesc B: Klasyfikacja dla podzbiorow o rozmiarze 5
Accuracy dla k=5: 20.32%
Czesc B: Klasyfikacja dla podzbiorow o rozmiarze 10
Accuracy dla k=5: 24.25%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics.pairwise import cosine_similarity

# Parametry
batch_size = 128
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 100  # CIFAR-100

# Transformacje
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Wczytanie zbioru danych
trainset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

# Podział zbioru na A i B
half_classes = num_classes // 2
trainset_A_indices = np.where(np.array(trainset.targets) < half_classes)[0]
trainset_B_indices = np.where(np.array(trainset.targets) >= half_classes)[0]

testset_A_indices = np.where(np.array(testset.targets) < half_classes)[0]
testset_B_indices = np.where(np.array(testset.targets) >= half_classes)[0]

trainloader_A = torch.utils.data.DataLoader(torch.utils.data.Subset(trainset, trainset_A_indices), batch_size=batch_size, shuffle=True)
trainloader_B = torch.utils.data.DataLoader(torch.utils.data.Subset(trainset, trainset_B_indices), batch_size=batch_size, shuffle=True)

testloader_A = torch.utils.data.DataLoader(torch.utils.data.Subset(testset, testset_A_indices), batch_size=batch_size, shuffle=False)
testloader_B = torch.utils.data.DataLoader(torch.utils.data.Subset(testset, testset_B_indices), batch_size=batch_size, shuffle=False)

# Definicja ekstraktora cech
class FeatureExtractor(nn.Module):
    def __init__(self, model):
        super(FeatureExtractor, self).__init__()
        self.feature_extractor = nn.Sequential(*list(model.feature_extractor.children())[:-1])

    def forward(self, x):
        x = self.feature_extractor(x)
        return x.view(x.size(0), -1)

# Wczytanie modelu
model = CNN(num_classes=50)
model.load_state_dict(torch.load("model.pth"))
model.to(device)
model.eval()
feature_extractor_A = FeatureExtractor(model).to(device)

def extract_features(loader, feature_extractor):
    features, labels = [], []
    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            feat = feature_extractor(inputs).cpu().numpy()
            features.append(feat)
            labels.append(targets.cpu().numpy())
    return np.vstack(features), np.hstack(labels)

train_features_A, train_labels_A = extract_features(trainloader_A, feature_extractor_A)
test_features_A, test_labels_A = extract_features(testloader_A, feature_extractor_A)

train_features_B, train_labels_B = extract_features(trainloader_B, feature_extractor_A)
test_features_B, test_labels_B = extract_features(testloader_B, feature_extractor_A)

def sample_subset(features, labels, samples_per_class):
    sampled_features, sampled_labels = [], []
    unique_classes = np.unique(labels)
    for cls in unique_classes:
        indices = np.where(labels == cls)[0]
        chosen_indices = np.random.choice(indices, min(samples_per_class, len(indices)), replace=False)
        sampled_features.append(features[chosen_indices])
        sampled_labels.append(labels[chosen_indices])
    return np.vstack(sampled_features), np.hstack(sampled_labels)

def cosine_similarity_classification(train_features, train_labels, test_features, test_labels):
    similarities = cosine_similarity(test_features, train_features)
    predicted_labels = train_labels[np.argmax(similarities, axis=1)]
    accuracy = np.mean(predicted_labels == test_labels)
    print(f'Accuracy: {accuracy * 100:.2f}%')

# Klasyfikacja dla A
for subset_size in [1, 5, 10]:
    sampled_train_features_A, sampled_train_labels_A = sample_subset(train_features_A, train_labels_A, subset_size)
    print(f'Czesc A: Klasyfikacja dla podzbiorow o rozmiarze {subset_size}')
    cosine_similarity_classification(sampled_train_features_A, sampled_train_labels_A, test_features_A, test_labels_A)

# Klasyfikacja dla B
for subset_size in [1, 5, 10]:
    sampled_train_features_B, sampled_train_labels_B = sample_subset(train_features_B, train_labels_B, subset_size)
    print(f'Czesc B: Klasyfikacja dla podzbiorow o rozmiarze {subset_size}')
    cosine_similarity_classification(sampled_train_features_B, sampled_train_labels_B, test_features_B, test_labels_B)

Files already downloaded and verified
Files already downloaded and verified


<ipython-input-9-fa3d3ff110af>:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model.pth"))  # Wczytujemy wytrenowane wagi


Czesc A: Klasyfikacja dla podzbiorow o rozmiarze 1
Accuracy: 24.68%
Czesc A: Klasyfikacja dla podzbiorow o rozmiarze 5
Accuracy: 35.82%
Czesc A: Klasyfikacja dla podzbiorow o rozmiarze 10
Accuracy: 40.68%
Czesc B: Klasyfikacja dla podzbiorow o rozmiarze 1
Accuracy: 12.88%
Czesc B: Klasyfikacja dla podzbiorow o rozmiarze 5
Accuracy: 20.12%
Czesc B: Klasyfikacja dla podzbiorow o rozmiarze 10
Accuracy: 21.16%
